### Объединение таблиц в pandas.

Привет, чат! В этом проекте вы столкнетесь сразу с несколькими таблицами и я считаю не помешает вспомнить, как их грамотно объединять.

Итак, вновь возьмем знакомую нам таблицу с данными по зарплатам аналитиков в условных единицах.

In [24]:
import pandas as pd

df = pd.DataFrame(data={'Имя': ['Анна', 'Сергей', 'Алексей', 'Сергей', 'Екатерина'], 'Фамилия': ['Егорова', 'Тищенко', 'Маевский', 'Пеньков', 'Никонова'] ,'Январь': [1000, 1300, 800, 1100, 2000], 'Февраль': [1100, 1250, 750, 1100, 1800], 
     'Март': [950, 1320, 900, 1200, 1950]})
df

,Имя,Фамилия,Январь,Февраль,Март
0,Анна,Егорова,1000,1100,950
1,Сергей,Тищенко,1300,1250,1320
2,Алексей,Маевский,800,750,900
3,Сергей,Пеньков,1100,1100,1200
4,Екатерина,Никонова,2000,1800,1950


Допустим, у нас есть отдельная таблица, где хранятся данные по накопленному стажу в месяцах для сотрудников:

In [25]:
status = pd.DataFrame(data={'Фамилия': ['Егорова', 'Тищенко', 'Маевский', 'Самсонов', 'Никонова'], 'Стаж':[6,18,12,4,10]})
status

,Фамилия,Стаж
0,Егорова,6
1,Тищенко,18
2,Маевский,12
3,Самсонов,4
4,Никонова,10


В таком виде данные хрянятся очень часто и вам постоянно придется объединять различные таблицы, чтобы получить полную картину.

Предположим, что у нас нет однофамильцев и фамилия однозначно определяет сотрудника. Было бы хорошо присоединить к общей таблице данные о стаже и проанализировать все сразу. Для этого используется метод `merge`. Давайте применим его, указав фамилию в качестве столбца для объединения и посмотрим, что получится:

In [26]:
df.merge(status, on='Фамилия')

,Имя,Фамилия,Январь,Февраль,Март,Стаж
0,Анна,Егорова,1000,1100,950,6
1,Сергей,Тищенко,1300,1250,1320,18
2,Алексей,Маевский,800,750,900,12
3,Екатерина,Никонова,2000,1800,1950,10


Отлично, получили таблицу с полными данными, но, похоже, кого-то потеряли! Действительно, пропал Сергей Пеньков. Думаю, все догадались, что это произошли оттого, что его нет во второй таблице. Но обратите внимание, некоего Самсонова из таблицы status в итоговом результате тоже нет.  
Это все результат выбра метода объединения, который задается в аргументе *how*. По умолчанию он имеет значение *inner*. То есть мы проводим внутреннее объединение - в итоговой таблице окажутся только те сотрудники, для которых нашлись одинаковые фамилии и в правой и в левой таблицах.  
Но что если мы не хотим терять Сергея и готовы мириться с пропуском на месте его стажа? Применим тип объединения *left*:

In [27]:
df.merge(status, on='Фамилия', how = 'left')

,Имя,Фамилия,Январь,Февраль,Март,Стаж
0,Анна,Егорова,1000,1100,950,6.0
1,Сергей,Тищенко,1300,1250,1320,18.0
2,Алексей,Маевский,800,750,900,12.0
3,Сергей,Пеньков,1100,1100,1200,NaN
4,Екатерина,Никонова,2000,1800,1950,10.0


Отлично, все сотрудники из начальной таблицы на месте. А Сергей, возможно, еще не наработал релевантного стажа, но это не повод исключать его из анализа.  
*При выборе типа объединения **left** берутся все строки из левой таблицы и им ищутся соответствия в правой. Если соответствия нет, ставится пропуск.*

Аналогично, если мы хотим увидеть всех сотрудников, по которым есть данные по стажу (то есть из правой таблицы) применим тип объединения *right*:

In [28]:
df.merge(status, on='Фамилия', how = 'right')

,Имя,Фамилия,Январь,Февраль,Март,Стаж
0,Анна,Егорова,1000.0,1100.0,950.0,6
1,Сергей,Тищенко,1300.0,1250.0,1320.0,18
2,Алексей,Маевский,800.0,750.0,900.0,12
3,Екатерина,Никонова,2000.0,1800.0,1950.0,10
4,NaN,Самсонов,NaN,NaN,NaN,4


Видим, что прочих данных по сотруднику Самсонову нет, но проанализировать его стаж мы сможем. А Сергей Пеньков снова пропал, так как данные по его стажу отсутствуют.

И, наконец, можно сделать объединение по всем имеющимся строкам, применив метод объединения *outer*:

In [29]:
df.merge(status, on='Фамилия', how = 'outer')

,Имя,Фамилия,Январь,Февраль,Март,Стаж
0,Анна,Егорова,1000.0,1100.0,950.0,6.0
1,Сергей,Тищенко,1300.0,1250.0,1320.0,18.0
2,Алексей,Маевский,800.0,750.0,900.0,12.0
3,Сергей,Пеньков,1100.0,1100.0,1200.0,NaN
4,Екатерина,Никонова,2000.0,1800.0,1950.0,10.0
5,NaN,Самсонов,NaN,NaN,NaN,4.0


Все имеющиеся в двух таблицах сотрудники на месте, но есть пропуски в данных.

Как видим, все очень просто, но будьте аккуратны, неверно выбрав метод объединения, можно потерять данные.

Также, нужно быть аккуратными и следить, чтобы ключи были уникальными, иначе результат может быть неожиданным.  
Добавим в датафрейм имена и попробуем провести объединение по ним

In [39]:
status = pd.DataFrame(data={'Имя': ['Анна', 'Сергей', 'Алексей', 'Сергей', 'Екатерина'],'Фамилия': ['Егорова', 'Тищенко', 'Тищенко', 'Пеньков', 'Никонова'], 'Стаж':[6,18,12,4,10]})
status

,Имя,Фамилия,Стаж
0,Анна,Егорова,6
1,Сергей,Тищенко,18
2,Алексей,Тищенко,12
3,Сергей,Пеньков,4
4,Екатерина,Никонова,10


In [40]:
df.merge(status, on='Имя', how = 'outer')

,Имя,Фамилия_x,Январь,Февраль,Март,Фамилия_y,Стаж
0,Анна,Егорова,1000,1100,950,Егорова,6
1,Сергей,Тищенко,1300,1250,1320,Тищенко,18
2,Сергей,Тищенко,1300,1250,1320,Пеньков,4
3,Сергей,Пеньков,1100,1100,1200,Тищенко,18
4,Сергей,Пеньков,1100,1100,1200,Пеньков,4
5,Алексей,Маевский,800,750,900,Тищенко,12
6,Екатерина,Никонова,2000,1800,1950,Никонова,10


Как видим, появились новые строки и столбцы: каждому Сергею из первой таблицы сопоставляются два Сергея из второй. Чтобы этого избежать, можно воспользоваться составным ключом:

In [41]:
df.merge(status, on=['Имя','Фамилия'], how = 'outer')

,Имя,Фамилия,Январь,Февраль,Март,Стаж
0,Анна,Егорова,1000.0,1100.0,950.0,6.0
1,Сергей,Тищенко,1300.0,1250.0,1320.0,18.0
2,Алексей,Маевский,800.0,750.0,900.0,NaN
3,Сергей,Пеньков,1100.0,1100.0,1200.0,4.0
4,Екатерина,Никонова,2000.0,1800.0,1950.0,10.0
5,Алексей,Тищенко,NaN,NaN,NaN,12.0


Такой код обеспечит корректную работу даже в случае, если в фирме будут однофамильцы - строки задаются однозначно парами *Имя-Фамилия*.

### Бонус: генерация списков и словарей.

Также, в этом спринте вам стали встречаться примеры с генерацией списков и не все понимают, что это за дивная конструкция. Давайте коротко разберемся с ней.

Допустим, у нас есть список зарплат наших аналитиков за январь без учета налогов:

In [42]:
jan_list = df['Январь'].tolist()
jan_list

[1000, 1300, 800, 1100, 2000]

Теперь мы хотим посчитать, сколько они реально получат, после вычета НДФЛ. Пройдем по списку, вычтя из каждой зарплаты налоги:

In [43]:
net_salaries = []  # создадим список с "чистыми" зарплатами


for salary in jan_list:
    net = salary * 0.87 #считаем чистую зарплату за вычетом тринадцати процентов
    net_salaries.append(net)  #добавляем в список
    
net_salaries

[870.0, 1131.0, 696.0, 957.0, 1740.0]

Отлично, мы заплатили налоги и можем спать спокойно! Однако на подобное действие приходится выполнять довольно часто и каждый раз писать для этого цикл неудобно. Для упрощения создания новых списков применяется эта самая *геренеция списков*.  
Давайте попорбуем сделать то же самое с её применением:

In [44]:
net_salaries = [salary * 0.87 for salary in jan_list]
net_salaries

[870.0, 1131.0, 696.0, 957.0, 1740.0]

Получили тот же самый результат, но в одну короткую строку! Выражение `for salary in jan_list` говорит о том, что мы проходим циклом по списку `jan_list`, на каждой итерации сохраняя очередной элемент в переменной `salary`. Далее, в левой части выражения, делаем с этим элементом, что хотим: умножаем, делим, применяем любые функции. На выходе получаем новый список. О том, что мы хотим получить именно список, говорят квадратные скобки, в которые мы взяли выражение.  
Однако подобным образом мы можем сгеренировать и словарь. Давайте создадим на основе `jan_list` словарь, в котором ключами будут базовые зарплаты, а значениями "чистые", за вычетом налогов:

In [45]:
{salary:salary * 0.87 for salary in jan_list}

{1000: 870.0, 1300: 1131.0, 800: 696.0, 1100: 957.0, 2000: 1740.0}

Как видим, для генерации словаря мы берем выражение в фигурные скобки, а в левой части указываем ключи и значения через двоеточие.  
Ну и напоследок, давайте добавим в наш генератор условие. Например, нас интересут только заралаты ниже 1200:

In [46]:
{salary:salary * 0.87 for salary in jan_list if salary < 1200}

{1000: 870.0, 800: 696.0, 1100: 957.0}

Видим, что фильтр отработал корректно. Согласен, первое время немного кипит мозг от перестановки привычного порядка, но немного привыкнув, вы поймете, что на самом деле это очень удобно и читаемо.

На этом на сегодня все, спасибо всем, кто дочитал! 👍